In [1]:
import requests
import re
import json
from bs4 import BeautifulSoup


Y_MOVIE_URL = 'https://tw.movies.yahoo.com/movie_thisweek.html'

# 以下網址後面加上 "/id=MOVIE_ID" 即為該影片各項資訊
Y_INTRO_URL = 'https://tw.movies.yahoo.com/movieinfo_main.html'  # 詳細資訊
Y_PHOTO_URL = 'https://tw.movies.yahoo.com/movieinfo_photos.html'  # 劇照
Y_TIME_URL = 'https://tw.movies.yahoo.com/movietime_result.html'  # 時刻表


def get_web_page(url):
    resp = requests.get(url)
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text


def get_movies(dom):
    soup = BeautifulSoup(dom, 'lxml')
    movies = []
    rows = soup.find_all('div', 'movie_intro_info_r')
    for row in rows:
        movie = dict()
        #movie['expectation'] = row.find('div', 'leveltext').span.text.strip()
        #movie['ch_name'] = row.find('div', 'release_movie_name').a.text.strip()
        movie['ch_name'] = row.find('h1').string.strip('<h1>')
        movie['eng_name'] = row.find('h3').string.strip('<h3>')
        typenum =  row.find('div', 'level_name_box').find_all('a', class_ = "gabtn")
        type1 = []
        for movie_t in typenum:
            type1.append(movie_t.string.strip('\n').strip(' ').strip('\n'))
        movie['type'] = type1
        movie['director'] = row.find('div', 'movie_intro_list').string.strip()
        movie['release_date'] = row.span.text.strip('上映日期：')
        movie['intro'] = soup.find('div', 'gray_infobox_inner').span.text.strip()
        #movie['eng_name'] = row.find('div', 'release_movie_name').find('div', 'en').a.text.strip()
        #movie['movie_id'] = get_movie_id(row.find('div', 'release_movie_name').a['href'])
        #movie['poster_url'] = row.parent.find_previous_sibling('div', 'release_foto').a.img['src']
        #movie['release_date'] = get_date(row.find('div', 'release_movie_time').text)
        #movie['intro'] = row.find('div', 'release_text').text.replace(u'詳全文', '').strip()
        #trailer_a = row.find_next_sibling('div', 'release_btn color_btnbox').find_all('a')[1]
        #movie['trailer_url'] = trailer_a['href'] if 'href' in trailer_a.attrs.keys() else ''
        movies.append(movie)
    return movies


def get_date(date_str):
    # e.g. "上映日期：2017-03-23" -> match.group(0): "2017-03-23"
    pattern = '\d+-\d+-\d+'
    match = re.search(pattern, date_str)
    if match is None:
        return date_str
    else:
        return match.group(0)


def get_movie_id(url):
    # 20180515: URL 格式有變, e.g., 'https://movies.yahoo.com.tw/movieinfo_main/%E6%AD%BB%E4%BE%8D2-deadpool-2-7820.html
    # e.g., "https://tw.rd.yahoo.com/referurl/movie/thisweek/info/*https://tw.movies.yahoo.com/movieinfo_main.html/id=6707"
    #       -> match.group(0): "/id=6707"
    try:
        movie_id = url.split('.html')[0].split('-')[-1]
    except:
        movie_id = url
    return movie_id


def get_trailer_url(url):
    # e.g., 'https://tw.rd.yahoo.com/referurl/movie/thisweek/trailer/*https://tw.movies.yahoo.com/video/美女與野獸-最終版預告-024340912.html'
    return url.split('*')[1]


def get_complete_intro(movie_id):
    page = get_web_page(Y_INTRO_URL + '/id=' + movie_id)
    if page:
        soup = BeautifulSoup(page, 'html5lib')
        infobox = soup.find('div', 'gray_infobox_inner')
        print(infobox.text.strip())


def main():
    for i in range(5):
        url = "https://movies.yahoo.com.tw/movieinfo_main/" + str(i)
        page = get_web_page(url)
        if page:
            movies = get_movies(page)
            for movie in movies:
                print(movie)
                # get_complete_intro(movie["movie_id"])
                with open('movie.json', 'w', encoding='utf-8') as f:
                    json.dump(movies, f, indent=2, sort_keys=True, ensure_ascii=False)


if __name__ == '__main__':
    main()

{'ch_name': '一世狂野', 'eng_name': 'Blow', 'type': ['劇情', '犯罪', '歷史/傳記'], 'director': '泰德戴米(Ted Demme)', 'release_date': '2001-10-12', 'intro': '喬治戎格一生都在追求所謂的美國夢，也就是享受美好富裕的生活，但是他卻不願像他父親那樣一輩子都只是個出賣勞力的建築工人。於是他搬到陽光明媚的加州，靠著販賣大麻賺錢，起初，他販毒只是為了享受自由自在的生活，但是當他野心越來越大，他的勢力也日益坐大之際，卻在此時被捕入獄。 他在牢裡認識一個能言善道，自稱熟識哥倫比亞販毒集團的牢友狄亞哥，他出獄後果真把當時勢力最大的毒梟艾斯科巴介紹給喬治認識，艾斯科巴計畫將古柯鹼大量引進美國的迪斯可舞廳，希望能引領一股吸毒狂歡的風潮。除了毒品供應商之外，狄亞哥也介紹了一個美艷又狂野的女人瑪莎給喬治，他們瘋狂相愛，之後馬莎還替他生下一個可愛的女兒克莉絲汀娜，也是喬治一生的最愛。喬治很快就靠著販毒發大財，他還得買一棟大房子專門存放每天賺進來的大把鈔票，但是日進斗金卻整天提心吊膽的生活卻讓喬治開始省思，到底他要繼續過著揮霍富裕的生活，還是為了自己心愛的女兒應該轉性投資正當的事業？可是這時聯邦調查局的探員，也開始盯上毒源禍首的喬治……'}
{'ch_name': '玩命關頭', 'eng_name': 'The Fast and the Furious', 'type': ['動作', '劇情', '犯罪', '懸疑/驚悚'], 'director': '羅伯柯恩(Rob Cohen)', 'release_date': '2001-10-13', 'intro': '唐米尼杜洛托是洛城街頭賽車界的老大哥，他身邊有一群忠心耿耿的手下，他白天忙著組裝高性能跑車，晚上則是開著他的愛車，動輒以一次一萬美元的賭注和別人軋車。布萊恩也渴望接受極速的挑戰，他對自己的駕駛技術很有信心，但是在旁觀者的眼中他只是一個菜鳥，他開了一輛超炫的跑車想和唐老大一較高下，也希望得到他的青睞，當比賽結束，布萊恩輸得一塌塗地之後，警方接獲風聲前來取締，布萊恩在無意間從一名心狠手辣的幫派份子強尼手中救了唐老大一命，於是他就被納入唐老大的權力核心，唐老大的妹妹蜜雅也對布萊恩產生好感，但是他們都不知道布萊恩